In [126]:
import numpy as np
import pandas as pd
import sklearn
import os
import itertools

In [127]:
source_file = "./data/combined_data.csv"
destination = "./data"

df = pd.read_csv(source_file)
df.head()

,seq,enrichment,class
0,GGGTGCAA,0.471220,0
1,CGGGTGCA,0.468569,0
2,ATTGCACC,0.434169,0
3,CCGGGTGC,0.433263,0
4,CGCACCCG,0.406588,0


In [128]:
df["seq"].value_counts()

seq
GGGTGCAA    1
GGATACCA    1
ATGTATCC    1
AAACGGTG    1
CCCGTAAA    1
           ..
ATCGTGGC    1
AACAAAAA    1
CGATGATC    1
AAAGAGGG    1
CCCGATCG    1
Name: count, Length: 14681, dtype: int64

In [129]:
# Hard labels for sequence to class

In [130]:
class_truth = df.groupby('seq')['class'].agg(lambda x: x.mode()[0]).to_dict()

## k-mer Nucleotide Frequency

In [131]:
def generate_kmers(k):
    base_pairs = list("ACGT")
    kmers = itertools.combinations_with_replacement(base_pairs, k)
    return kmers

# N = 3 max atm, paper does N=4
for k in range(1, 4):
    kmers = generate_kmers(k)
    
    for kmer in kmers:
        kmer = "".join(kmer)
        df[f"count_{kmer}"] = df["seq"].apply(lambda x : x.count(kmer))

## k-Spaced Nucleotide Pair Frequency

In [132]:
pairs = list(generate_kmers(2))

def count_kspaced(seq, pair, k):
    count = 0
    for i in range(len(seq) - 2 - k):
        if seq[i] == pair[0] and seq[i+k+1] == pair[1]:
            count += 1
    return count

#paper does k=0-4, but k=1-4 works without redudent feature
for k in range(1, 5):
    for p in pairs:
        df[f"count_{p[0] + k * '.' + p[1]}"] = df["seq"].apply(lambda x : count_kspaced(x, p, k))

## Nucleotide Physicochemical Property

In [133]:
properties = ["h-bond", "func-group", "ring-struct"]
bps = ["AT", "AC", "AG"]

for p, bp in zip(properties, bps):
    for i in range(8):
        df[f"{p}_{i}"] = df["seq"].apply(lambda x : x[i] in bp)        

C:\Users\myang\AppData\Local\Temp\ipykernel_32324\451582947.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{p}_{i}"] = df["seq"].apply(lambda x : x[i] in bp)


In [134]:
#defragment frame
df = df.copy()

## Pseudo k-Tuple Nucleotide Composition

In [173]:
#Write genes to fafsa file
file_path = "./data/8mers.fasta"
bp = list("ACGT")
with open(file_path, 'w') as file:
    for seq in df['seq']:
        file.write(f">{seq2int(seq)}\n")
        file.write(f"{seq}\n")

In [172]:
def seq2int(seq):
    i = 0
    for c in list(seq):
        i *= 4
        i += list("ACGT").index(c)
    return i

def int2seq(num):
    seq = ""
    for i in range(8):
        c = num % 4
        num /= 4
        seq = list("ACGT")[int(c)] + seq
    return seq
        

### Downloading mathfeatures

$ docker pull bio21061993/mathfeature:latest

$ docker run -it --name mathfeature-terminal bio21061993/mathfeature bash

$ git clone https://github.com/Bonidia/MathFeature.git MathFeature-Terminal

$ cd MathFeature-Terminal

### Creating PseKNC from fafsa file


$ python3.7 methods/PseKNC.py -i 8mers.fasta -o output6.csv -l 1 -x files/propNames-DNA-k2.txt -xp files/propValues-DNA-k2.txt -seq 1 -t 2 -k 2 -j 5 -w 1.0 -s 2

In [166]:
#Read and map files:
#PseKNC as created by MathFeatures with lambda = 5, weight = 1
PseKNC = pd.read_csv("./data/output5.csv")
PseKNC = PseKNC.drop(columns='label')
PseKNC["seq"] = PseKNC["nameseq"].apply(lambda x : int2seq(int(x)))
PseKNC = PseKNC.drop(columns='nameseq')

df = pd.merge(df, PseKNC, on='seq', how='inner')

## Electron−Ion Interaction Pseudopotentials of Trinucleotide.

In [170]:
df.columns[50:]

Index(['count_A..T', 'count_C..C', 'count_C..G', 'count_C..T', 'count_G..G',
       'count_G..T', 'count_T..T', 'count_A...A', 'count_A...C', 'count_A...G',
       'count_A...T', 'count_C...C', 'count_C...G', 'count_C...T',
       'count_G...G', 'count_G...T', 'count_T...T', 'count_A....A',
       'count_A....C', 'count_A....G', 'count_A....T', 'count_C....C',
       'count_C....G', 'count_C....T', 'count_G....G', 'count_G....T',
       'count_T....T', 'h-bond_0', 'h-bond_1', 'h-bond_2', 'h-bond_3',
       'h-bond_4', 'h-bond_5', 'h-bond_6', 'h-bond_7', 'func-group_0',
       'func-group_1', 'func-group_2', 'func-group_3', 'func-group_4',
       'func-group_5', 'func-group_6', 'func-group_7', 'ring-struct_0',
       'ring-struct_1', 'ring-struct_2', 'ring-struct_3', 'ring-struct_4',
       'ring-struct_5', 'ring-struct_6', 'ring-struct_7', 'pseknc-0',
       'pseknc-1', 'pseknc-2', 'pseknc-3', 'pseknc-4', 'pseknc-5', 'pseknc-6',
       'pseknc-7', 'pseknc-8', 'pseknc-9', 'pseknc-10', '

## Train Test split

In [174]:
from sklearn.model_selection import train_test_split


X = df.copy()
y = X["class"]
X = X.drop(columns=["class", "enrichment"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [175]:
X_train.columns

Index(['seq', 'count_A', 'count_C', 'count_G', 'count_T', 'count_AA',
       'count_AC', 'count_AG', 'count_AT', 'count_CC',
       ...
       'pseknc-11', 'pseknc-12', 'pseknc-13', 'pseknc-14', 'pseknc-15',
       'pseknc-16', 'pseknc-17', 'pseknc-18', 'pseknc-19', 'pseknc-20'],
      dtype='object', length=120)

## Feature selection
Don't forget to do train test split first

In [176]:
#Paper uses XGBoost, ANOVA, Chi2, and LASSO
# from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.inspection import permutation_importance

In [177]:
X_train_numeric = X_train.drop(columns="seq")

In [178]:
#Feature selection from gradient boosting
clf = HistGradientBoostingClassifier().fit(X_train_numeric, y_train)

In [179]:
#Permutation-based feature selection, direct impurity based selection is misleading when many columns
result = permutation_importance(clf, X_train_numeric, y_train, n_repeats=20, random_state=42)

In [180]:
support = result.importances_mean > np.mean(result.importances_mean)
print(support)

[False False False  True False False False False False  True False False
 False False False False False  True False False False  True False  True
 False  True  True  True False False False False False False False  True
 False False False False False False  True False False False False False
 False  True False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False  True  True  True False False  True False  True  True  True
  True False  True  True  True  True  True  True  True  True False]


In [181]:
X_train_numeric.columns[support][3]

'count_AGG'

In [182]:
print(["seq"] + list(X_train_numeric.columns[support]))

['seq', 'count_T', 'count_CG', 'count_AAT', 'count_AGG', 'count_ATT', 'count_CCG', 'count_CCT', 'count_CGG', 'count_A.C', 'count_G.T', 'count_C..G', 'pseknc-0', 'pseknc-1', 'pseknc-2', 'pseknc-5', 'pseknc-7', 'pseknc-8', 'pseknc-9', 'pseknc-10', 'pseknc-12', 'pseknc-13', 'pseknc-14', 'pseknc-15', 'pseknc-16', 'pseknc-17', 'pseknc-18', 'pseknc-19']


In [183]:
X_train = X_train[["seq"] + list(X_train_numeric.columns[support])]
X_test = X_test[["seq"] + list(X_train_numeric.columns[support])]

### Saving to file

In [184]:
#Write genes to fafsa file
file_path = "./features/"
X_train.to_csv(f"{file_path}X_train.csv")
X_test.to_csv(f"{file_path}X_test.csv")
y_train.to_csv(f"{file_path}y_train.csv")
y_test.to_csv(f"{file_path}y_test.csv")